In [ ]:
# License: BSD
# Author: Sasank Chilamkurthy

from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torch.utils import data
import random
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import pandas as pd
import time
import os
import copy

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

plt.ion()   # interactive mode


In [ ]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}
data_dir = '/content/drive/My Drive/ProjetoIC/Brodatz_trainval_corroido1'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")



In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title('predicted: {}'.format(class_names[preds[j]]))
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

In [ ]:
#Finetuning the convnett
model_ft = models.resnet50(pretrained=True)
num_ftrs = model_ft.fc.in_features
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
model_ft.fc = nn.Linear(num_ftrs, len(class_names))

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.002, momentum=0.8)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=9, gamma=0.6)

model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=25)

plt.ioff()
plt.show()

Epoch 0/24
----------
train Loss: 4.8437 Acc: 0.0101
val Loss: 4.1498 Acc: 0.0608

Epoch 1/24
----------
train Loss: 4.3166 Acc: 0.0417
val Loss: 3.4092 Acc: 0.1216

Epoch 2/24
----------
train Loss: 3.9059 Acc: 0.0845
val Loss: 2.9353 Acc: 0.2489

Epoch 3/24
----------
train Loss: 3.5736 Acc: 0.1295
val Loss: 2.7264 Acc: 0.2635

Epoch 4/24
----------
train Loss: 3.2255 Acc: 0.1847
val Loss: 2.6047 Acc: 0.3198

Epoch 5/24
----------
train Loss: 2.9910 Acc: 0.2342
val Loss: 1.8900 Acc: 0.4268

Epoch 6/24
----------
train Loss: 2.8114 Acc: 0.2725
val Loss: 2.0507 Acc: 0.4414

Epoch 7/24
----------
train Loss: 2.6987 Acc: 0.2815
val Loss: 1.8765 Acc: 0.4887

Epoch 8/24
----------
train Loss: 2.5377 Acc: 0.3243
val Loss: 1.9124 Acc: 0.4505

Epoch 9/24
----------
train Loss: 2.2268 Acc: 0.3975
val Loss: 1.5017 Acc: 0.5698

Epoch 10/24
----------
train Loss: 2.1067 Acc: 0.4324
val Loss: 1.3955 Acc: 0.5721

Epoch 11/24
----------
train Loss: 2.0169 Acc: 0.4313
val Loss: 1.2542 Acc: 0.6160

Ep

In [ ]:
#ConvNet as fixed feature extractor

model_conv = models.resnet50(pretrained=True)
for param in model_conv.parameters():
    param.requires_grad = False

# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, len(class_names))

model_conv = model_conv.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that only parameters of final layer are being optimized as
# opposed to before.
optimizer_conv = optim.SGD(model_conv.fc.parameters(), lr=0.01, momentum=0.5)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.4)

model_conv = train_model(model_conv, criterion, optimizer_conv,
                         exp_lr_scheduler, num_epochs=25)

plt.ioff()
plt.show()

Epoch 0/24
----------
train Loss: 5.1358 Acc: 0.0123
val Loss: 5.1875 Acc: 0.0090

Epoch 1/24
----------
train Loss: 4.8172 Acc: 0.0325
val Loss: 4.4672 Acc: 0.0224

Epoch 2/24
----------
train Loss: 4.4839 Acc: 0.0694
val Loss: 3.9208 Acc: 0.0907

Epoch 3/24
----------
train Loss: 4.2795 Acc: 0.0817
val Loss: 4.4787 Acc: 0.0437

Epoch 4/24
----------
train Loss: 4.0755 Acc: 0.1209
val Loss: 3.7247 Acc: 0.1265

Epoch 5/24
----------
train Loss: 4.0112 Acc: 0.1310
val Loss: 3.4891 Acc: 0.1736

Epoch 6/24
----------
train Loss: 3.9013 Acc: 0.1389
val Loss: 3.2290 Acc: 0.2128

Epoch 7/24
----------
train Loss: 3.5298 Acc: 0.1960
val Loss: 2.9067 Acc: 0.3281

Epoch 8/24
----------
train Loss: 3.4020 Acc: 0.2161
val Loss: 2.9712 Acc: 0.2576

Epoch 9/24
----------
train Loss: 3.3613 Acc: 0.2240
val Loss: 2.9247 Acc: 0.2844

Epoch 10/24
----------
train Loss: 3.3471 Acc: 0.2105
val Loss: 2.8379 Acc: 0.2934

Epoch 11/24
----------
train Loss: 3.3354 Acc: 0.2262
val Loss: 2.7769 Acc: 0.3247

Ep